In [ ]:
import os
import torch

import pyvdirs.dirs as dirs
import sys
sys.path.insert(0, dirs.SYSTEM_HOME)
sys.path.insert(0, os.path.join(dirs.SYSTEM_HOME, "ToyExample"))
from socket import gethostname

from ToyExample.toy_example import do_test
import pyvtools.text as vtext

In [22]:
test_batch_size = 2**14
test_seed = 7

# series = ["18_Statistics", "19_ACIDParams", "21_Repetitions", "23_NormalizedLogits"]
series = ["23_NormalizedLogits"]

get_path = lambda series : os.path.join(dirs.MODELS_HOME, "ToyExample", series)

In [23]:
host_id = gethostname()
other_hosts = vtext.filter_by_string_must(list(dirs.check_directories_file().keys()), [host_id,"else"], must=False)

In [24]:
test_results = {}

for s in series:

    series_path = get_path(s)
    contents = os.listdir(series_path)
    folders = [c for c in contents if os.path.isdir(os.path.join(series_path, c))]
    folders = vtext.filter_by_string_must(folders, ["Failed", "Old"], must=False)
    log_files = ["log_"+f+".txt" for f in folders]
    assert all([os.path.isfile(os.path.join(series_path, f)) for f in log_files]), "Some logs have not been found"

    test_results[s] = {}
    for folder, log_file in zip(folders, log_files):

        log_filepath = os.path.join(series_path, log_file)

        files = os.listdir(os.path.join(series_path, folder))
        net_file = vtext.filter_by_string_must(files, "learner")[0]
        EMA_file = "".join(net_file.split("learner"))

        net_filepath = os.path.join(series_path, folder, net_file)
        EMA_filepath = os.path.join(series_path, folder, EMA_file)

        with open(log_filepath, "r") as f:
            acid = False
            for i, line in enumerate(f):
                if "ACID = True" in line:
                    acid = True
                if "Guide model loaded from" in line or i>70: 
                    break
        if "Guide model loaded from" in line:
            guide_line = line
            guide_filepath = guide_line.split("Guide model loaded from ")[-1].split("\n")[0]
            for h in other_hosts:
                guide_filepath = guide_filepath.replace(dirs.check_directories_file()[h]["models_home"], dirs.MODELS_HOME)
        else:
            guide_filepath = None

        folder_results = do_test(
            net_filepath, ema_path=EMA_filepath, guide_path=guide_filepath, acid=acid, 
            classes='A', P_mean=-2.3, P_std=1.5, sigma_max=5, depth_sep=5,
            n_samples=test_batch_size, batch_size=test_batch_size, test_outer=True,
            guidance_weight=3,
            seed=test_seed, generator=None,
            device=torch.device('cuda'))
        
        test_results[s][folder] = folder_results

2025-05-14 14:07:28 | INFO     | Seed = 7
2025-05-14 14:07:28 | INFO     | Seed = 7
2025-05-14 14:07:28 | INFO     | Number of test epochs = 1
2025-05-14 14:07:28 | INFO     | Number of test epochs = 1
2025-05-14 14:07:28 | INFO     | Test batch size = 16384
2025-05-14 14:07:28 | INFO     | Test batch size = 16384
2025-05-14 14:07:28 | INFO     | Number of test samples = 16384
2025-05-14 14:07:28 | INFO     | Number of test samples = 16384
2025-05-14 14:07:28 | WARNING  | Model loaded from /media/vale/QuickData/Vale/Training/models/SCID/ToyExample/23_NormalizedLogits/ACID/iter4096learner.pkl
2025-05-14 14:07:28 | WARNING  | Model loaded from /media/vale/QuickData/Vale/Training/models/SCID/ToyExample/23_NormalizedLogits/ACID/iter4096learner.pkl
2025-05-14 14:07:28 | WARNING  | Model loaded from /media/vale/QuickData/Vale/Training/models/SCID/ToyExample/23_NormalizedLogits/ACID/iter4096learner.pkl
2025-05-14 14:07:28 | WARNING  | EMA model loaded from /media/vale/QuickData/Vale/Training/